In [ ]:
import pandas as pd
import numpy as np
from numpy import set_printoptions
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
import missingno
import re

from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, auc, roc_curve, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, recall_score, f1_score, precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import ensemble
from sklearn.metrics import confusion_matrix as cm
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from warnings import filterwarnings
from scipy.stats import norm

Kütüphanelerimizi import ediyoruz.

In [ ]:
tablets = pd.read_csv("../input/tablet/tablet.csv").copy()

Elimdeki csv dosyasını kopyalayıp "tablets" değişkenine atadım.

In [ ]:
print("Boyutu: ",tablets.ndim, "'dir.'")
print("Şekli: ",tablets.shape)
print("Eleman sayısı: ",tablets.size)

Tablomuz 2 boyutludur. 2000 adet tablet ve bunlara ait 20 özellik bulunmaktadır yani 40000 adet elemana sahibiz.

In [ ]:
tablets.info()

In [ ]:
tablets.head(10)
#Normal = 0 Pahalı =1 Ucuz=2 Çok ucuz = 3

İlk 10 verimizi gösterdik.

In [ ]:
tablets.sample(5)

Rastgele 5 adet veri çektim.

In [ ]:
sns.violinplot(x="FiyatAraligi",y="RAM",data = tablets)

Viyolin grafikten görüceğimiz üzere ram, büyük ölçekte fiyata etki ediyor. Tabi daha sonra ayrıntılı olarak tüm özelliklerin fiyata olan etkilerini inceleyeceğiz.

In [ ]:
tablets.isnull().sum()

Bazı verilerimin eksik olduğunu görüyorum.

In [ ]:
missingno.matrix(tablets,figsize=(15,10))

In [ ]:
tablets.describe().T

Veri Miktarı, ortalamaları, standart sapmaları, minimum-maksimum değerleri ve diğerleri. Minimum değerlerine bakıcak olursak çözünürlük yüksekliği 0 olanın çözünürlük genişliği olduğunu görüyoruz. Böyle bir şeyin mümkün olmadığı için fiyat ortalamalarına göre çözünürlük yükseklikleri 0 olanları düzeltiyoruz.

In [ ]:
tablets[tablets["CozunurlukYükseklik"]==0]

In [ ]:
tablets.groupby(["FiyatAraligi"]).describe()["CozunurlukYükseklik"]

In [ ]:
tablets.loc[662,"CozunurlukYükseklik"] = 666
tablets.loc[856,"CozunurlukYükseklik"] = 744

Ucuz olan 666 iken pahalı olan 744 olduğu için bu değerlere yeni değerlerini atıyoruz.

In [ ]:
def eksik_deger_tablosu(tablets):
    eksik_deger = tablets.isnull().sum()
    eksik_deger_yuzde = 100*tablets.isnull().sum()/len(tablets)
    eksik_deger_tablo = pd.concat([eksik_deger,eksik_deger_yuzde], axis=1)
    eksik_deger_tablo_son = eksik_deger_tablo.rename(columns = {0: 'Eksik değerler',1:'% değeri'})
    return eksik_deger_tablo_son

Eksik değerlerimizin, verilerin yüzde kaçını oluşturduğunu görmek için fonksiyon oluşturduk.

In [ ]:
eksik_deger_tablosu(tablets)

Ön kamera verilerinin %0.25'i ve Ram verilerinin %0.60'ının eksik olduğunu görüyoruz.

In [ ]:
sns.scatterplot(x="OnKameraMP", y="ArkaKameraMP", data = tablets);

Eksik veriler bulunmakta iken oluşan grafiğimiz.

In [ ]:
OnKameraLoc = tablets[tablets["OnKameraMP"].isnull()].index
OnKameraLoc

In [ ]:
RamLoc = tablets[tablets["RAM"].isnull()].index
RamLoc

Ön kameradaki ve Ramdeki eksik verilerin lokasyonlarını buldum ve değişkenlere atadım.

In [ ]:
tablets.describe().T

In [ ]:
tablets.loc[OnKameraLoc, "OnKameraMP"] = 4
tablets.loc[RamLoc, "RAM"] = 2116
tablets.isnull().sum()

%0.25 ve %0.6 'lık eksik verileri ortalama değerler ile kapattık.

Şimdi string değişkenlere sahip olan özellikleri nümerik,integer değerlere çevireceğiz.

In [ ]:
label_encoder = preprocessing.LabelEncoder()

In [ ]:
tablets["FiyatAraligi"].head(10)

In [ ]:
tablets["Bluetooth"] = label_encoder.fit_transform(tablets["Bluetooth"])
tablets["CiftHat"] = label_encoder.fit_transform(tablets["CiftHat"])
tablets["4G"] = label_encoder.fit_transform(tablets["4G"])
tablets["3G"] = label_encoder.fit_transform(tablets["3G"])
tablets["Dokunmatik"] = label_encoder.fit_transform(tablets["Dokunmatik"])
tablets["WiFi"] = label_encoder.fit_transform(tablets["WiFi"])

j=0
for i in tablets["FiyatAraligi"]:
    if tablets.loc[j,"FiyatAraligi"] == "Çok Ucuz":
        tablets.loc[j,"FiyatAraligi"] = 0
    elif tablets.loc[j,"FiyatAraligi"] == "Ucuz":
        tablets.loc[j,"FiyatAraligi"] = 1
    elif tablets.loc[j,"FiyatAraligi"] == "Normal":
        tablets.loc[j,"FiyatAraligi"] = 2
    elif tablets.loc[j,"FiyatAraligi"] == "Pahalı":
        tablets.loc[j,"FiyatAraligi"] = 3
    j+=1

tablets.head(8)

Verileri sayısallaştırdık. Renk ve fiyat aralığı dışındakiler 1 ve 0 lardan oluşmakta. Bu yüzden fiyatların doğrusal şekilde artabilmesi için doğru sırayla atama yaptık.

In [ ]:
sns.scatterplot(x="Renk", y="FiyatAraligi", data = tablets, color = "green");

In [ ]:
sns.barplot(x = "Renk", y= "FiyatAraligi", data = tablets);

Renkleri nümerik ifadeye çevirmeyi yani sayısallaştırmayı düşünüyoruz fakat fiyatlara olan etkisinin pek olmadığını görüp siliyoruz.

In [ ]:
#tablets["Renk"] = pd.Categorical(tablets["Renk"])
#dummies = pd.get_dummies(tablets["Renk"], prefix = "Renk")


#tablets = pd.concat([tablets, dummies], axis = 1)

Renkleri bu şekilde sayısallaştırabilirdik.

In [ ]:
tablets.drop("Renk",axis=1,inplace=True)
#tablets.head(2)

Burda asıl renk kolonumuzu kaldırıyoruz.

In [ ]:
tablets.median(),'======================================',tablets.mean()

Medyanları ve ortalamalarına bakarak her birinin dağılımı hakkında yorum yapabiliriz.

In [ ]:
tablets.corr()

Aralarındaki korelasyonlara bakacak olursak herhangi iki parametre arasında doğrusal bir korelasyon olduğunu söyleyemem. Çünkü hepsi 0 a çok yakın sayılar.

In [ ]:
corr = tablets.corr()
sns.heatmap(corr,
           xticklabels = corr.columns.values,
           yticklabels = corr.columns.values);

Korelasyon tablosunda gözümden kaçan verileri ısı haritasında farkettik. Ön kamera-Arka kamera arasındaki korelasyon, Çözünürlük yükseklik-Çözünürlük genişlik arasındaki korelasyon ve 4G-3G arasındaki korelasyon diğerlerine nazaran daha yüksek korelasyona sahiptir.

In [ ]:
tablets.groupby("OnKameraMP")["ArkaKameraMP"].apply(lambda x: np.mean(x))

Ön kamera ile Arka kamera arasındaki korelasyonu gözlemlemek istiyoruz.

In [ ]:
sns.scatterplot(x="OnKameraMP", y="ArkaKameraMP", data = tablets);

Grafikten de görüleceği üzere  aralarında doğrusal bir ilişki bulunmakta. 4 noktasındaki fazladan noktalar bizim doldurduğumuz eksik veriler olduğu için bu iki noktayı ayrıca düzeltiyoruz.

In [ ]:
tablets[tablets["ArkaKameraMP"]<tablets["OnKameraMP"]]

Altta kalan 2 değer bunlarmış.

In [ ]:
tablets[(tablets["OnKameraMP"] == 4)&(tablets["FiyatAraligi"] == 0)&(tablets["4G"] == 1)&(tablets["CekirdekSayisi"] == 3)]

In [ ]:
tablets.loc[792,"ArkaKameraMP"] = 10
tablets[(tablets["FiyatAraligi"] == 0)&(tablets["4G"] == 0)&(tablets["CekirdekSayisi"] == 5)&(tablets["ArkaKameraMP"] == 3)]

Ön kamera arka kameradan hiçbir zaman büyük olamadığı için 726. indistekine benzer özelliği bulup onun değerini aktardık.

In [ ]:
tablets.loc[726,"OnKameraMP"] = 2

In [ ]:
sns.lmplot(x="OnKameraMP", y="ArkaKameraMP", data = tablets)

In [ ]:
sns.scatterplot(x="RAM", y="FiyatAraligi", data = tablets);

Ram'in fiyatla olan ilişkisinin doğrusal olduğunu yorumlayabiliriz.

In [ ]:
sns.jointplot(x = tablets["OnKameraMP"], y = tablets["ArkaKameraMP"], kind = "kde");

Jointplot kullanarak verilerin çoğunlukta bulunduğu aralıkları görebiliyoruz. 

In [ ]:
sns.countplot(tablets["FiyatAraligi"]);

Fiyat aralıklarına bakarak dengeli bir dağılım olduğunu söyleyebiliriz.

### Fiyata Etki Eden Değişkenlere bakıyoruz.

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "BataryaGucu", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "Bluetooth", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "MikroislemciHizi", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "CiftHat", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "OnKameraMP", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "4G", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "DahiliBellek", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "Kalinlik", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "Agirlik", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "CekirdekSayisi", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "ArkaKameraMP", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "CozunurlukYükseklik", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "CozunurlukGenislik", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "RAM", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "BataryaOmru", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "3G", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "Dokunmatik", data = tablets);

In [ ]:
sns.barplot(x = "FiyatAraligi", y= "WiFi", data = tablets);

## Regresyon aşamasına geçiyoruz

In [ ]:
x = tablets.drop("FiyatAraligi",axis=1)
y = tablets["FiyatAraligi"]

In [ ]:
lm1 = LinearRegression()
model = lm1.fit(x,y)

In [ ]:
model.score(x,y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 42)

%85 ve %15 olarak ayırıp modelimizi test ediyoruz.

In [ ]:
lm2 = LinearRegression()
model2 = lm2.fit(x_train, y_train)

İkinci modelimizi tekrar eğitiyoruz

In [ ]:
model2.score(x_train, y_train)

In [ ]:
y_pred = model2.predict(x_test)
y_pred

In [ ]:
r2_score(y_train, model2.predict(x_train))

In [ ]:
cross_val_score(model2, x_train, y_train, cv = 10, scoring = "r2")

In [ ]:
cross_val_score(model2, x_train, y_train, cv = 10, scoring = "r2").mean()

model.score() için eğitim verisini verdiğimizde %91.0704 başarı elde ediyorduk, halbuki 10 farklı parça üzerinde aynı işlemi yaptığımızda ortalama %90.45275477952819'a düştü. Çünkü böylelikle test edilmeyen hiçbir gözlem kalmadı. Tüm gözlemler test ve train olmak üzere yer değiştirebilme şansı buldu.

In [ ]:
j=0
for i in tablets["FiyatAraligi"]:
    if tablets.loc[j,"FiyatAraligi"] == 0:
        tablets.loc[j,"FiyatAraligi"] = "Çok Ucuz"
    elif tablets.loc[j,"FiyatAraligi"] == 1:
        tablets.loc[j,"FiyatAraligi"] = "Ucuz"
    elif tablets.loc[j,"FiyatAraligi"] == 2:
        tablets.loc[j,"FiyatAraligi"] = "Normal"
    elif tablets.loc[j,"FiyatAraligi"] == 3:
        tablets.loc[j,"FiyatAraligi"] = "Pahalı"
    j+=1
    
x = tablets.drop("FiyatAraligi",axis=1)
y = tablets["FiyatAraligi"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 42)

Burada Gaussian fonksiyonunu kullanmak için string ifadelere geri döndürdüm.

In [ ]:
nb = GaussianNB()
nb_model = nb.fit(x,y)

In [ ]:
y_pred = nb_model.predict(x)

Veri çerçevemizdeki tüm gözlemleri kontrol edip y_pred değişkenine atayalım.

In [ ]:
accuracy_score(y,y_pred)

%81.15 doğruluk oranı çıktığını görüyoruz.

In [ ]:
cross_val_score(nb_model,x,y,cv=10).mean()

10 parçaya bölüp ayrı ayrı tüm veri setini gözlemledik. Daha sonra bunların ortalamasını aldık.

In [ ]:
print(classification_report(y,y_pred))

In [ ]:
PrecisionScore = precision_score(y, y_pred, average="weighted")
PrecisionScore

In [ ]:
RecallScore = recall_score(y, y_pred, average="weighted")
RecallScore

In [ ]:
F1Score = f1_score(y, y_pred, average="weighted")
F1Score

In [ ]:
test = SelectKBest(score_func = f_classif,k="all")
fit = test.fit(x,y)
fit

In [ ]:
set_printoptions(precision=3)
for indis, skor in enumerate(fit.scores_):
    print(skor," = ", x.columns[indis])

Her özniteliğin fiyatla aralarında olan skorlarını yazdırdık. 18 tane özniteliği tek tek değerlendirmek yerine en çok etki edenleri değerlendirerek doğruluk oranımızı arttırmayı hedefliyoruz.

In [ ]:
x = tablets.drop(["FiyatAraligi","WiFi"], axis = 1)

In [ ]:
nb = GaussianNB()
nb_model = nb.fit(x,y)
y_pred = nb_model.predict(x)

In [ ]:
accuracy_score(y,y_pred)

## Desicion Tree

### Gini ile:

In [ ]:
x = tablets.drop(["FiyatAraligi"], axis=1)
y = tablets["FiyatAraligi"]

In [ ]:
x

In [ ]:
y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

In [ ]:
cart_grid = {"max_depth": range(1,20),
            "min_samples_split" : range(2,50)}

In [ ]:
cart = DecisionTreeClassifier()
cart_cv = GridSearchCV(cart, cart_grid, cv = 10, n_jobs = -1, verbose = 2)

In [ ]:
cart_cv_model = cart_cv.fit(x_train, y_train)

In [ ]:
print("En iyi parametreler : " + str(cart_cv_model.best_params_))
print("En iyi skor : " + str(cart_cv_model.best_score_))

In [ ]:
cart = DecisionTreeClassifier(max_depth = 7, min_samples_split = 3)
cart_tuned = cart.fit(x_train, y_train)

In [ ]:
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG
from graphviz import Source
from IPython.display import display

graph = Source(tree.export_graphviz(cart, out_file = None, feature_names = x.columns, filled = True))
display(SVG(graph.pipe(format = 'svg')))

In [ ]:
y_pred = cart_tuned.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred)

Daha önce hiç görmediği x_test verileri için bulunduğu tahminlere göre doğruluk skorunu görüntüleyelim.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

Karmaşıklık matrisini görüntüleyelim.

In [ ]:
cross_val_score(cart_tuned, x_test, y_test, cv = 10)

Modeli farklı gözlemlerle kombinleyerek tekrardan skorunu hesaplayalım.

In [ ]:
cross_val_score(cart_tuned, x, y, cv = 10).mean()

10 skorun ortalamasına bakalım.

In [ ]:
print(classification_report(y_test, y_pred))

Modelin karar ağacını oluştururken root node için "Ram"e dayanarak karar verdiğini görüyoruz. Özniteliklerin karar ağacına göre önem sıralamasına bakalım.

In [ ]:
ranking = cart_tuned.feature_importances_
features = np.argsort(ranking)[::-1][:10]
columns = x.columns

plt.figure(figsize = (16, 9))
plt.title("Karar Ağacına Göre Özniteliklerin Önem Derecesi", y = 1.03, size = 18)
plt.bar(range(len(features)), ranking[features], color="green", align="center")
plt.xticks(range(len(features)), columns[features], rotation=80)
plt.show()

### Entropy ile:

In [ ]:
x = tablets.drop(["FiyatAraligi"], axis=1)
y = tablets["FiyatAraligi"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=0.15,
                                                    random_state=42)

In [ ]:
cart_grid = {"max_depth": range(1,20),
            "min_samples_split" : range(2,50)}

In [ ]:
cart = DecisionTreeClassifier(criterion = "entropy")
cart_cv = GridSearchCV(cart, cart_grid, cv = 10, n_jobs = -1, verbose = 2)

In [ ]:
cart_cv_model = cart_cv.fit(x_train, y_train)

In [ ]:
print("En iyi parametreler : " + str(cart_cv_model.best_params_))
print("En iyi skor : " + str(cart_cv_model.best_score_))

%86 doğruluk oranı çıktı gayet güzel.

In [ ]:
cart = DecisionTreeClassifier(max_depth = 15, min_samples_split = 10)
cart_tuned = cart.fit(x_train, y_train)

In [ ]:
graph = Source(tree.export_graphviz(cart, out_file = None, feature_names = x.columns, filled = True))
display(SVG(graph.pipe(format = 'svg')))

In [ ]:
y_pred = cart_tuned.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

In [ ]:
cross_val_score(cart_tuned, x_test, y_test, cv = 10)

In [ ]:
cross_val_score(cart_tuned, x, y, cv = 10).mean()

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
ranking = cart_tuned.feature_importances_
features = np.argsort(ranking)[::-1][:10]
columns = x.columns

plt.figure(figsize = (16, 9))
plt.title("Karar Ağacına Göre Özniteliklerin Önem Derecesi", y = 1.03, size = 18)
plt.bar(range(len(features)), ranking[features], color="green", align="center")
plt.xticks(range(len(features)), columns[features], rotation=80)
plt.show()

## K En Yakın Komşu Algoritması

In [ ]:
knn_params = {"n_neighbors": np.arange(2,50)}

In [ ]:
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, knn_params, cv = 3)
knn_cv.fit(x_train, y_train)

In [ ]:
print("En iyi skor: " + str(knn_cv.best_score_))
print("En iyi parametreler: " + str(knn_cv.best_params_))

In [ ]:
knn = KNeighborsClassifier(13)
knn_tuned = knn.fit(x_train, y_train)

In [ ]:
y_pred = knn_tuned.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
score_list = []

for each in range(2,50):
    knn2 = KNeighborsClassifier(n_neighbors = each)
    knn2.fit(x_train,y_train)
    score_list.append(knn2.score(x_test, y_test))
    
plt.plot(range(2,50),score_list)
plt.xlabel("k değerleri")
plt.ylabel("doğruluk skoru")
plt.show()

In [ ]:
cross_val_score(knn_tuned, x_test, y_test, cv = 10)

In [ ]:
cross_val_score(knn_tuned, x_test, y_test, cv = 10).mean()